In [78]:
import pandas as pd
import datetime as dt
import numpy as np
import csv

from numpy import sin, cos, sqrt

from astropy.time import Time

import pdir

import os

## Import

In [79]:
# Script input

# FILENAME = "horizons_results_earth_2019-2020.txt"
# FILENAME = "horizons_results_earth_2019-2039.txt"
FILENAME = "horizons_results_earth_2019-2262.txt"

# FILENAME = "horizons_results_mars_2019-2020.txt"
# FILENAME = "horizons_results_mars_2019-2039.txt"
# FILENAME = "horizons_results_mars_2019-2262.txt"

limited_timespan = True  # Set false if more years than 2019-2261 are wanted, but dates will not be converted

Assumed format of filename:

`horizons_results_[BODY]_[START_YEAR]-[END-YEAR].txt`

where the dates of START_YEAR (closed interval) and END_YEAR (open interval) are assumed to be 01-01 00:00:00 (i.e. January 1st 00:00:00).

Due to Timestamp conversion limitations, we don't allow dates above 2262-01-01:
```python
print(pd.Timestamp.min)  # 1677-09-21 00:12:43.145225
print(pd.Timestamp.max)  # 2262-04-11 23:47:16.854775807
```

If more years are needed, re-write script to now use `pd.to_datetime`, i.e. convert date to pandas type Timestamp.

NOTE:
Horizon limits the number of rows such that startin from 2019, 2265 is the maximum year anyway.
Also ephemerides are [available](https://ssd.jpl.nasa.gov/eph_spans.cgi?id=A) in the following time spans:

- **Earth:** B.C. 9998-Mar-20 to A.D. 9999-Dec-31
- **Mars:** 1600-Jan-01 to 2500-Jan-04 

In [80]:
# Change CDW (current working directiory)

FILE_PATH = "../orbsim/r4b_3d/ephemerides/"

cwd = os.getcwd()
in_correct_cwd = 'code' + FILE_PATH[2:-1] == cwd[-30:] # check if last part of cwd is '/code/orbsim/r4b_3d'

if not in_correct_cwd:    
    os.chdir(FILE_PATH)
    cwd = os.getcwd()

print(cwd)

/Users/gandalf/Dropbox/repositories/letomes/code/orbsim/r4b_3d/ephemerides


In [81]:
# Derived variables

_, _, BODY, DATE_RANGE = FILENAME.split('.')[0].split('_')
START_YEAR, END_YEAR = DATE_RANGE.split('-')
OUTPUT = "{}_{}.csv".format(BODY, DATE_RANGE)  # output filename

with open(FILENAME, 'r') as file:
    for i, line in enumerate(file):
        if line == "$$SOE\n":
            SOE = i + 1
        if line == "$$EOE\n":
            EOE = i + 1

FOOTER = 117
EOD = EOE + FOOTER

print("SOE: {}".format(SOE))
print("EOE: {}".format(EOE))
print("FOOTER: {}".format(FOOTER))
print("EOD: {}".format(EOD))

print("BODY: {}".format(BODY))
print("DATE_RANGE: {}".format(DATE_RANGE))
print("START_YEAR: {}".format(START_YEAR))
print("END_YEAR: {}".format(END_YEAR))
print("OUTPUT: {}".format(OUTPUT))
print("SOE: {}".format(SOE))

SOE: 68
EOE: 88824
FOOTER: 117
EOD: 88941
BODY: earth
DATE_RANGE: 2019-2262
START_YEAR: 2019
END_YEAR: 2262
OUTPUT: earth_2019-2262.csv
SOE: 68


In [82]:
# Check if date interval is valid

if int(START_YEAR) < 1678:
    raise ValueError("Ephemerides end date must be in a year no earier than 1678 (01-01), otherwise pd.to_datetime fails down below")

if int(END_YEAR) > 2262:
    raise ValueError("Ephemerides end date must be in a year no later than 2262 (01-01), otherwise pd.to_datetime fails down below")

In [83]:
# Read in raw txt file

df = pd.read_csv(FILENAME,
                 skiprows=[x for x in range(0,SOE) if x!=SOE-3], # skip all rows until SOE except for headers
                 skipfooter=FOOTER,
                 engine='python',
                 error_bad_lines=False,
                 quoting=csv.QUOTE_NONE)

df.rename(columns=lambda x: x.strip(), inplace=True)  # strip whitespace from headers
df.drop('Unnamed: 26', axis=1, inplace=True)  # dropped erroneous 'n.p.' column

df

,Date__(UT)__HR:MN:SC.fff,Date_________JDUT,,.1,R.A._(ICRF/J2000.0),DEC_(ICRF/J2000.0),dRA*cosD,d(DEC)/dt,SN.ang,SN.dist,...,delta,deldot,VmagSn,VmagOb,PlAng,ObsEcLon,ObsEcLat,GlxLon,GlxLat,Tru_Anom
0,2019-Jan-01 00:00:00.000,2458484.5,,,100.873526,23.060100,151.8959,-17.6227,96.62,0.0,...,0.983311,-0.016109,30.29273,30.29273,-0.00330,99.995349,-0.002696,191.381921,8.640234,358.1927
1,2019-Jan-02 00:00:00.000,2458485.5,,,101.977771,22.979791,151.9257,-17.4809,96.56,0.0,...,0.983304,-0.008699,30.29513,30.29513,-0.00328,101.014827,-0.002672,191.901697,9.521088,359.0326
2,2019-Jan-03 00:00:00.000,2458486.5,,,103.080725,22.891843,151.9525,-17.3333,96.51,0.0,...,0.983301,-0.001588,30.29699,30.29699,-0.00327,102.034382,-0.002640,192.424192,10.401239,359.8247
3,2019-Jan-04 00:00:00.000,2458487.5,,,104.182248,22.796298,151.9761,-17.1799,96.45,0.0,...,0.983302,0.005304,30.29825,30.29825,-0.00326,103.053990,-0.002600,192.949663,11.280589,0.5832
4,2019-Jan-05 00:00:00.000,2458488.5,,,105.282199,22.693202,151.9965,-17.0210,96.39,0.0,...,0.983307,0.012058,30.29889,30.29889,-0.00325,104.073628,-0.002555,193.478370,12.159035,1.3218
5,2019-Jan-06 00:00:00.000,2458489.5,,,106.380440,22.582604,152.0134,-16.8564,96.33,0.0,...,0.983316,0.018751,30.29890,30.29890,-0.00324,105.093269,-0.002506,194.010579,13.036476,2.0539
6,2019-Jan-07 00:00:00.000,2458490.5,,,107.476837,22.464559,152.0271,-16.6864,96.26,0.0,...,0.983329,0.025459,30.29829,30.29829,-0.00323,106.112888,-0.002454,194.546560,13.912804,2.7919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88748,2261-Dec-26 00:00:00.000,2547232.5,,,90.546126,23.404354,151.6217,-18.6445,97.01,0.0,...,0.983863,-0.121997,30.28003,30.28003,0.00918,90.501189,-0.033987,186.651088,0.397821,346.3293
88749,2261-Dec-27 00:00:00.000,2547233.5,,,91.655828,23.396669,151.6415,-18.5562,96.98,0.0,...,0.983795,-0.115118,30.27965,30.27966,0.00918,91.519646,-0.033888,187.157452,1.281512,347.0041


### NOTE OUTLIER
271 September 29th, crazy declanation value

## Extract & rename relevant columns, export

In [84]:
# Extract and renam relevant columns, then export

eph = pd.DataFrame()
# eph['MJD'] = [Time(x).mjd for x in list(eph['date'].astype(str))]  # using astropy to convert
eph['MJD'] = df['Date_________JDUT'] - 2400000.5  # using definition   (https://en.wikipedia.org/wiki/Julian_day#Variants)
eph['date'] =  pd.to_datetime(df['Date__(UT)__HR:MN:SC.fff'])

eph['day'] = list(eph['MJD']) - eph['MJD'][0]
eph.set_index('day', inplace=True)

eph['r'] =  df['r']
eph['theta'] = 90 - df['DEC_(ICRF/J2000.0)']
eph['phi'] = df['R.A._(ICRF/J2000.0)']

rs = list(eph['r'])
thetas = np.radians(list(eph['theta']))
phis = np.radians(list(eph['phi']))
eph['x'] = rs * sin(thetas) * cos(phis)
eph['y'] = rs * sin(thetas) * sin(phis)
eph['z'] = rs * cos(thetas)
# eph['r2'] = sqrt(eph['x']**2 + eph['y']**2 + eph['z']**2)  # sanity check

eph.to_csv(OUTPUT)

pd.set_option('max_row', 15)
eph

,MJD,date,r,theta,phi,x,y,z
day,,,,,,,,
0.0,58484.0,2019-01-01,0.983311,66.939900,100.873526,-0.170672,0.888496,0.385160
1.0,58485.0,2019-01-02,0.983304,67.020209,101.977771,-0.187873,0.885562,0.383888
2.0,58486.0,2019-01-03,0.983301,67.108157,103.080725,-0.205017,0.882352,0.382497
3.0,58487.0,2019-01-04,0.983302,67.203702,104.182248,-0.222098,0.878866,0.380986
4.0,58488.0,2019-01-05,0.983307,67.306798,105.282199,-0.239109,0.875105,0.379357
5.0,58489.0,2019-01-06,0.983316,67.417396,106.380440,-0.256047,0.871070,0.377608
6.0,58490.0,2019-01-07,0.983329,67.535441,107.476837,-0.272904,0.866763,0.375742
...,...,...,...,...,...,...,...,...
88748.0,147232.0,2261-12-26,0.983863,66.595646,90.546126,-0.008606,0.902874,0.390808
